In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
from collections import deque

In [2]:
# calibrate the camera
nx = 9
ny = 6
objpoints = []
imgpoints = []

objp = np.zeros((nx * ny, 3), dtype=np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

for impath in glob('camera_cal/*.jpg'):
    # Find the chessboard corners
    gray = cv2.cvtColor(cv2.imread(impath), cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret is True:
        imgpoints.append(corners)
        objpoints.append(objp)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 2. use a function from one of the quizes, skip params
def cal_undistort(img):
    # Use cv2.calibrateCamera() and cv2.undistort()
    return cv2.undistort(img, mtx, dist, None, mtx)

In [3]:
# util functions to combine sobel kernel gradient with saturation thresholding
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    x = int(orient == 'x')
    y = int(orient == 'y')
    abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, x, y, ksize=sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    mask = (scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[mask] = 1
    return binary_output

def hls_select(img, thresh=(0, 255)):
    S = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,2]
    mask = (S > thresh[0]) & (S <= thresh[1])
    binary_output = np.zeros_like(S)
    binary_output[mask] = 1
    return binary_output

In [4]:
# combine all preprocessing
def preprocess(img):
    tmp = cal_undistort(img)
    hls = hls_select(tmp, thresh=(140,255))
    mag = abs_sobel_thresh(tmp, orient='x',
                       sobel_kernel=5,
                       thresh=(40,255))
    combined = np.zeros_like(hls)
    combined[(hls == 1) | (mag == 1)] = 1
    return combined

In [47]:
class Lane():
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    def __init__(self):
        self.fit = None # 2nd order coefficients
        self.radius_m = None # radius of curvature
        self.offset = None # offset in meters from left edge
        self.smooth_hist = deque([], 8)
        
    def update(self, fit_poly2, fit_poly2_cr, drop_x2_thresh=0.0001):
        # updates lane instance if sanity checks are passed   
        if self.fit is not None:
            if abs(self.fit[0] - fit_poly2[0]) > drop_x2_thresh:
                # print("skip frame, x2 diff {}".format(self.fit[0] - fit_poly2[0]))
                return
        self.fit = self.__avg_poly(fit_poly2)
        self.offset = fit_poly2_cr[2]
        self.__update_radius(fit_poly2_cr)
    
    def __avg_poly(self, new_poly):
        self.smooth_hist.appendleft(new_poly)
        return np.mean(self.smooth_hist, axis=0)
    
    def __update_radius(self, fit_cr):
        # Radius of curvature
        y_eval = 719
        self.radius_m = ((1 + (2*fit_cr[0]*y_eval*self.ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])

class Processor():
    def __init__(self):
        self.M = None
        self.Minv = None
        self.lane_l = Lane()
        self.lane_r = Lane()
        
    def center_offset(self, centerx):
        lx = self.lane_l.offset
        rx = self.lane_r.offset
        return lx + (rx - lx) / 2 - centerx * Lane.xm_per_pix
    
    def frame(self, img_orig):
        
        img = preprocess(img_orig)
        img_size = (img.shape[1], img.shape[0])
        
        if self.M is None:
            # warp using the transformation matrix M, cache it
            warp_src = np.float32(
                [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
                [((img_size[0] / 6) - 20), img_size[1]],
                [(img_size[0] * 5 / 6) + 20, img_size[1]],
                [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
            warp_dst = np.float32(
                [[(img_size[0] / 4), 0],
                [(img_size[0] / 4), img_size[1]],
                [(img_size[0] * 3 / 4), img_size[1]],
                [(img_size[0] * 3 / 4), 0]])
            self.M = cv2.getPerspectiveTransform(warp_src, warp_dst)
            self.Minv = np.linalg.inv(self.M)

        warped = cv2.warpPerspective(img, self.M, img_size, flags=cv2.INTER_NEAREST)
    
        # search using sliding windows
        left_fit, left_fit_cr, right_fit, right_fit_cr = self.__slide_window_fit(warped)
        self.lane_l.update(left_fit, left_fit_cr)
        self.lane_r.update(right_fit, right_fit_cr)
        
        return self.__draw_fit_unwarped(img_orig, warped)
        
    def __draw_fit_unwarped(self, original, warped):
        # Generate x and y values for plotting
        left_fit = self.lane_l.fit
        right_fit = self.lane_r.fit
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.Minv, (warped.shape[1], warped.shape[0])) 
        # Combine the result with the original image
        combined = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
        cv2.putText(combined,
                    "l_curve: {}m".format(str(round(self.lane_l.radius_m,2))),
                    (20,50),
                    cv2.FONT_HERSHEY_DUPLEX,
                    1,
                    (255,0,0))
        cv2.putText(combined,
                    "r_curve: {}m".format(str(round(self.lane_r.radius_m,2))),
                    (20,100),
                    cv2.FONT_HERSHEY_DUPLEX,
                    1,
                    (255,0,0))
        cv2.putText(combined,
                    "offset: {}m".format(str(round(self.center_offset(warped.shape[1] / 2),2))),
                    (20,150),
                    cv2.FONT_HERSHEY_DUPLEX,
                    1,
                    (255,0,0))
        return combined

    def __slide_window_fit(self, binary_warped, nwindows=9):
        histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
        
        # select where to begin search from the bottom half of the histogram
        x_midpoint = np.int(histogram.shape[0] / 2)
        leftx_base = np.argmax(histogram[:x_midpoint])
        rightx_base = np.argmax(histogram[x_midpoint:]) + x_midpoint
        
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*Lane.ym_per_pix, leftx*Lane.xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*Lane.ym_per_pix, rightx*Lane.xm_per_pix, 2)
        return left_fit, left_fit_cr, right_fit, right_fit_cr

In [48]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

p = Processor()

output = 'harder_challenge_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
clip = clip1.fl_image(p.frame)
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video harder_challenge_output.mp4
[MoviePy] Writing video harder_challenge_output.mp4


100%|█████████▉| 1199/1200 [04:59<00:00,  5.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_output.mp4 

CPU times: user 4min 42s, sys: 13.2 s, total: 4min 55s
Wall time: 5min 2s


In [50]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))